# Step 4: Evaluate the application

## 1. Run the app on our evaluation dataset

In [1]:
%run 01-llm-app-setup.ipynb

Repo card metadata block was not found. Setting CardData to empty.
100%|██████████| 5/5 [00:00<00:00, 1915.03it/s]


Parsing nodes:   0%|          | 0/5 [00:00<?, ?it/s]

Documents before chunking: 5
Documents after chunking: 35


/Users/akashsaravanan/Library/Caches/pypoetry/virtualenvs/genai-bootcamp-4wh1UwyX-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


 = Plain maskray = 
 
 The plain maskray or brown stingray ( Neotrygon annotata ) is a species of stingray in the family Dasyatidae . It is found in shallow , soft-bottomed habitats off northern Australia . Reaching 24 cm ( 9.4 in ) in width , this species has a diamond-shaped , grayish green pectoral fin disc . Its short , whip-like tail has alternating black and white bands and fin folds above and below . There are short rows of thorns on the back and the base of the tail , but otherwise the s


Generating embeddings:   0%|          | 0/35 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The plain maskray is found in the continental shelf of northern Australia from the Wellesley Islands in Queensland to the Bonaparte Archipelago in Western Australia, including the Gulf of Carpentaria and the Timor and Arafura Seas. There are also unsubstantiated reports that its range extends to southern Papua New Guinea.

Source: Plain maskray
Relevant Snippet: "The plain maskray inhabits the continental shelf of northern Australia from the Wellesley Islands in Queensland to the Bonaparte Archipelago in Western Australia, including the Gulf of Carpentaria and the Timor and Arafura Seas. There are unsubstantiated reports that its range extends to southern Papua New Guinea."


In [2]:
import pandas as pd

gen_dataset = pd.read_csv('generated_qa.csv')
# Running only on 5 items for speed
sample_size = 5

In [3]:
gen_dataset["answer"] = None
gen_dataset["contexts"] = None

for idx, item in gen_dataset.iloc[:sample_size].iterrows():
    result = openai_query_engine.query(item.question)
    gen_dataset.at[idx, "answer"] = result.response
    gen_dataset.at[idx, "contexts"] = [node.text for node in result.source_nodes]


In [4]:
gen_dataset.iloc[:sample_size]

,question,ground_truth,ground_truth_context,answer,contexts
0,What is the full Japanese title of Valkyria Ch...,Senjō no Valkyria 3: Unrecorded Chronicles (Ja...,= Valkyria Chronicles III = \n \n Senjō no Val...,The full Japanese title of Valkyria Chronicles...,[= Valkyria Chronicles III = \n \n Senjō no Va...
1,What is the main progression system in the game?,The player progresses through a series of line...,The player progresses through a series of line...,"The main progression system in ""Valkyria Chron...",[The player progresses through a series of lin...
2,What are the five classes of troops mentioned ...,"The five classes of troops are Scouts, Shocktr...",Troops are divided into five classes : Scouts ...,The five classes of troops mentioned in the te...,[Troops are divided into five classes : Scouts...
3,Who is the commanding officer of the Nameless?,Ramsey Crowe,"Hounded by both allies and enemies , and combi...",The commanding officer of the Nameless is Rams...,"[Hounded by both allies and enemies , and comb..."
4,Why was Valkyria Chronicles III developed for ...,The team wanted to refine the mechanics create...,"Like its predecessor , Valkyria Chronicles III...",Valkyria Chronicles III was developed for the ...,"[Like its predecessor , Valkyria Chronicles II..."


## 2. Run evaluation

This might take a few minutes. We call our pre-defined evaluation functions and also run ragas on our dataset.

In [5]:
%run 03-metrics-definition.ipynb

In [6]:
results_lst = []
 
for idx, row in gen_dataset.iloc[:sample_size].iterrows(): # Subsetting to make it go faster
    custom_eval_results = {
        "context_correctness": context_correctness(row["ground_truth_context"], row["contexts"]),
        "ground_truth_context_rank": ground_truth_context_rank(row["ground_truth_context"], row["contexts"]),
        "context_rougel_score": context_rougel_score(row["ground_truth_context"], row["contexts"]),
    }

    ragas_eval_results = evaluate_w_ragas(row)
    results_lst.append(custom_eval_results | ragas_eval_results)


results_df = pd.DataFrame(results_lst)



Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
results_df

,context_correctness,ground_truth_context_rank,context_rougel_score,context_precision,faithfulness,answer_correctness
0,True,0,1.0,1.0,1.0,0.988061
1,True,0,1.0,1.0,1.0,0.621889
2,True,0,1.0,1.0,1.0,0.989046
3,True,0,1.0,1.0,1.0,0.205459
4,True,0,1.0,1.0,1.0,0.725229


And that's it! We can aggregate these metrics to get a single number for each of them and we have a good evaluation of our model.